In [1]:
from models.swin_transformer_v2_cr import swin_v2_cr_tiny_ns_224


In [2]:
model = swin_v2_cr_tiny_ns_224(pretrained=True)


C:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [1]:
import torch


In [4]:
output = model(torch.randn((1, 3, 224, 224)))


In [7]:
output


tensor([[ 1.9532e-02,  5.4793e-02,  1.9980e-01,  1.6374e-01,  1.3614e-01,
          2.3772e-01,  1.3757e-01,  6.9767e-02,  1.2979e-01,  1.7200e-01,
          1.3954e-01,  1.7284e-01,  1.9191e-01,  7.0005e-02,  1.6745e-01,
          1.1273e-01,  1.6154e-01,  5.2753e-02,  1.9300e-01,  1.4329e-01,
          1.6648e-01,  1.9819e-01,  8.6066e-02,  1.6280e-01,  2.1150e-01,
          8.8097e-02,  1.7161e-01,  2.0532e-01,  1.1030e-01,  1.2649e-01,
          1.4704e-01,  8.5227e-02,  1.6203e-01,  1.2522e-01,  1.3478e-01,
          1.0711e-01,  8.1430e-02,  5.3488e-02,  2.1402e-01,  1.0009e-01,
          7.3824e-02,  2.1044e-01,  1.1900e-01,  3.3592e-03,  7.6803e-02,
          1.4379e-01,  1.1446e-01,  1.5589e-01,  8.6907e-02,  1.2359e-01,
          2.1268e-01, -6.5931e-03,  2.4438e-01,  2.0533e-01,  1.7634e-01,
          1.0542e-01,  1.6079e-01,  7.2592e-02,  2.7741e-01,  9.1539e-02,
          1.7666e-01,  1.7365e-01,  2.8749e-02,  8.2977e-02,  1.2555e-01,
          3.1942e-01,  1.2310e-01,  1.

In [ ]:
# See what's the actual loss function.
# See what's the actual



In [ ]:
# criterion = torch.nn.CrossEntropyLoss()

# it get a 1000 classes output, but I need it to be a sequence.


In [ ]:
#### use this formate to run: https://github.com/SwinTransformer/Swin-Transformer-Object-Detection/blob/master/mmdet/models/backbones/swin_transformer.py


In [ ]:
# [TODO] creating a swin transformer with fpn and fit it into current rcnn model.
# Write about DINO.


In [4]:
import torch, gc
import pandas as pd

from models.load import ModelSetup, create_model_from_setup
from models.build import Backbones
from utils.data import get_datasets, get_dataloaders

## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn'


In [5]:
gc.collect()
# torch.cuda.memory_summary(device=None, abbreviated=False)

use_gpu = torch.cuda.is_available()
device = "cuda" if use_gpu else "cpu"
print(f"This notebook will running on device: [{device.upper()}]")

if use_gpu:
    torch.cuda.empty_cache()


This notebook will running on device: [CUDA]


In [6]:
XAMI_MIMIC_PATH = "D:\XAMI-MIMIC"

use_iobb = True
io_type_str = "IoBB" if use_iobb else "IoU"

all_model_setups = [
    ModelSetup(
        name="custom_without_clinical_swin",
        use_clinical=False,
        use_custom_model=True,
        use_early_stop_model=True,
        backbone = Backbones.Swin,
    ),
    ModelSetup(
        name="custom_with_clinical_swin",
        use_clinical=True,
        use_custom_model=True,
        use_early_stop_model=True,
        backbone = Backbones.Swin,
    ),
]


In [7]:
model_setup = all_model_setups[0]


In [8]:
labels_cols = [
    "Enlarged cardiac silhouette",
    "Atelectasis",
    "Pleural abnormality",
    "Consolidation",
    "Pulmonary edema",
    #  'Groundglass opacity', # 6th disease.
]

dataset_params_dict = {
    "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
    "with_clinical": model_setup.use_clinical,
    "dataset_mode": "normal",
    "bbox_to_mask": True,
    "labels_cols": labels_cols,
}

detect_eval_dataset, train_dataset, val_dataset, test_dataset = get_datasets(
    dataset_params_dict=dataset_params_dict
)

train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
    train_dataset, val_dataset, test_dataset, batch_size=4
)


In [9]:
model = create_model_from_setup(
    labels_cols,
    model_setup,
    rpn_nms_thresh=0.3,
    box_detections_per_img=6,
    box_nms_thresh=0.2,
    rpn_score_thresh=0.0,
    box_score_thresh=0.05,
    clinical_input_channels=32,
    clinical_num_len=9,
    clinical_conv_channels=64,
    fuse_conv_channels=64,
)

model.to(device)


C:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


MultimodalMaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): SwinFPN(
    (swin): SwinTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
        (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (layers): ModuleList(
        (0): BasicLayer(
          (blocks): ModuleList(
            (0): SwinTransformerBlock(
              (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (attn): WindowAttention(
                (qkv): Linear(in_features=96, out_features=288, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=96, out_features=96, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (sof

In [13]:
from models.rcnn import multimodal_maskrcnn_resnet50_fpn

model = multimodal_maskrcnn_resnet50_fpn(
            num_classes=91,
            pretrained=True,
        )

In [13]:
data = next(iter(train_dataloader))
data = train_dataset.prepare_input_from_data(data, device)


In [15]:
data[0][0].shape


torch.Size([3, 3056, 2544])

In [17]:
images, clinical_num, clinical_cat, targets = data

images, targets = model.transform(images, targets)


In [20]:
images.tensors.shape

backbone_input = images.tensors


In [21]:
backbone_output = model.backbone(backbone_input)


In [23]:
backbone_output.keys()


odict_keys(['0', '1', '2', '3', 'pool'])

In [25]:
backbone_input.shape


torch.Size([4, 3, 512, 512])

In [24]:
from models.swin import FPN, SwinTransformer


In [ ]:
import torch.nn as nn
from collections import OrderedDict

class SwinFPN(nn.Moudle):
    def __init__(
        self, swin, fpn,
    ):
        self.swin = swin
        self.fpn = fpn

        super().__init__()

    def forward(self, x):
        x = self.swin(x)
        x = self.fpn(x)

        out = OrderedDict({})
        last_idx = len(x) - 1

        for i, out in enumerate(fpn_out):

            if i == last_idx:
                out["pool"] = out

            else:
                out[f"{i}"] = out

        return out


In [31]:
swin = SwinTransformer(pretrain_img_size=512)
swin.to(device)


SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): BasicLayer(
      (blocks): ModuleList(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path): Identity()
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU()
            (fc2): Linear(in_features

In [35]:
swin_out = swin(backbone_input)


In [52]:
len(swin_out)


4

In [36]:
swin_out[0].shape


torch.Size([4, 96, 128, 128])

In [37]:
swin_out[1].shape


torch.Size([4, 192, 64, 64])

In [40]:
swin_out[2].shape


torch.Size([4, 384, 32, 32])

In [38]:
swin_out[3].shape


torch.Size([4, 768, 16, 16])

In [61]:
fpn_args = {
    "in_channels": [96, 192, 384, 768],
    "out_channels": 256,
    "num_outs": 5,
}

In [62]:
fpn = FPN(**fpn_args)

fpn.to(device)


FPN(
  (lateral_convs): ModuleList(
    (0): ConvModule(
      (conv): Conv2d(96, 256, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): ConvModule(
      (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1))
    )
    (2): ConvModule(
      (conv): Conv2d(384, 256, kernel_size=(1, 1), stride=(1, 1))
    )
    (3): ConvModule(
      (conv): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1))
    )
  )
  (fpn_convs): ModuleList(
    (0): ConvModule(
      (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1): ConvModule(
      (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (2): ConvModule(
      (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (3): ConvModule(
      (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
)

In [45]:
fpn_out = fpn(swin_out)


In [57]:
from collections import OrderedDict

output_dict = OrderedDict({})

last_idx = len(fpn_out) - 1
for i, out in enumerate(fpn_out):

    if i == last_idx:
        output_dict["pool"] = out

    else:
        output_dict[f"{i}"] = out



In [58]:
output_dict


OrderedDict([('0',
              tensor([[[[ 2.3878e+00,  5.1937e+00,  5.2172e+00,  ...,  2.8210e+00,
                          2.4603e+00,  9.6247e-01],
                        [ 3.1254e+00,  3.3681e+00,  2.6483e+00,  ...,  8.7624e-01,
                          7.5655e-01, -2.3797e+00],
                        [ 3.1534e+00,  3.2985e+00,  2.5655e+00,  ...,  8.1721e-01,
                          7.6951e-01, -2.3369e+00],
                        ...,
                        [ 1.1191e+00,  4.8355e-01,  4.8206e-01,  ..., -1.0489e-02,
                          1.1449e-01, -1.7068e+00],
                        [ 1.0532e+00,  5.9708e-01,  5.9623e-01,  ...,  4.4973e-02,
                          2.6259e-01, -1.6755e+00],
                        [-1.1306e+00, -3.7275e-01, -3.7255e-01,  ...,  1.1625e-01,
                          1.6698e-01, -5.3386e-01]],
              
                       [[ 7.4508e-01,  4.0792e-01,  2.6861e+00,  ...,  2.0695e+00,
                          2.1318e+00,  7.25

In [49]:
for out in fpn_out:

    print(out.shape)


torch.Size([4, 256, 128, 128])
torch.Size([4, 256, 64, 64])
torch.Size([4, 256, 32, 32])
torch.Size([4, 256, 16, 16])
torch.Size([4, 256, 8, 8])


In [59]:
for k, v in output_dict.items():
    print(f"[{k}]: {v.shape}")


[0]: torch.Size([4, 256, 128, 128])
[1]: torch.Size([4, 256, 64, 64])
[2]: torch.Size([4, 256, 32, 32])
[3]: torch.Size([4, 256, 16, 16])
[pool]: torch.Size([4, 256, 8, 8])


In [50]:
for k, v in backbone_output.items():
    print(f"[{k}]: {v.shape}")


[0]: torch.Size([4, 256, 128, 128])
[1]: torch.Size([4, 256, 64, 64])
[2]: torch.Size([4, 256, 32, 32])
[3]: torch.Size([4, 256, 16, 16])
[pool]: torch.Size([4, 256, 8, 8])


In [53]:
type(backbone_output)


collections.OrderedDict

tensor([[[[ 6.4604e-01,  1.5593e+00,  1.7736e+00,  ...,  1.1103e+00,
            8.8171e-01,  2.5151e-02],
          [ 8.9652e-01,  1.9263e+00,  2.1528e+00,  ...,  1.1626e+00,
            8.5006e-01, -1.0418e-01],
          [ 6.3494e-01,  1.4152e+00,  1.8256e+00,  ...,  1.4602e+00,
            1.0784e+00,  1.9260e-01],
          ...,
          [-7.9515e-01, -1.3140e-01,  6.0951e-01,  ...,  3.4582e-01,
           -3.1437e-02, -1.2912e-01],
          [-8.5917e-01, -1.8239e-01,  5.9011e-01,  ...,  2.7922e-01,
           -1.3308e-01, -3.3160e-01],
          [-8.2335e-01, -7.7024e-01, -2.0603e-01,  ..., -3.6266e-01,
           -6.9379e-01, -8.0904e-01]],

         [[ 1.7511e+00,  1.4587e+00,  1.1996e+00,  ...,  1.3340e+00,
            1.7972e+00,  9.7162e-01],
          [ 2.0487e+00,  1.8958e+00,  1.6617e+00,  ...,  1.1746e+00,
            1.6106e+00,  7.5081e-01],
          [ 1.9860e+00,  2.1010e+00,  1.9134e+00,  ...,  8.7300e-01,
            1.2306e+00,  5.8559e-01],
          ...,
     